In [3]:
import gurobipy as grb
import numpy as np
import copy
import os
import CreateBaseModel as cbm

In [21]:
m = grb.read(r'.\test_model\SimpleIPModel_01.lp')

Read LP format model from file .\test_model\SimpleIPModel_01.lp
Reading time = 0.00 seconds
: 2 rows, 2 columns, 4 nonzeros


In [8]:
m.getVars()[0].vtype

'I'

# N. Wang
# 2024-06
# Branch and Bound Algorithm

In [81]:
def create_model(model_name='Model', model_var=add_var, model_CnO=add_CnO_simple_IP_model_01):
    created_model = grb.Model(model_name)
    
    # add var
    
    
    # add constraint and set objective
#     add_CnO_simple_IP_model_01(created_model, add_var(created_model))
    model_CnO(model=created_model, x_dict=model_var(created_model))
    
    created_model.optimize()
    save_ModelnSolution(created_model)
    return created_model
   
def add_var(model):
    var_index = [
    f'x_{i}'
    for i in range(1, 3)
    ]
    x_dict = model.addVars(var_index, vtype=grb.GRB.INTEGER, name= var_index)
    return x_dict
        
def add_CnO_simple_IP_model_01(model, x_dict):    
    model.addConstr(2 * x_dict['x_1'] + x_dict['x_2'] <=10, name= 'c1')
    model.addConstr(3 * x_dict['x_1'] + 6 * x_dict['x_2'] <=40, name='c2')
    model.setObjective(100 * x_dict['x_1'] + 150 * x_dict['x_2'], grb.GRB.MAXIMIZE)

def save_ModelnSolution(model, path=r'.\test_model'):
    model.write(os.path.join(path, model.ModelName+'.lp'))
    model.write(os.path.join(path, model.ModelName+'.sol'))


In [82]:
test = create_model(model_name='SimpleIPModel_01')

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22000.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x60e99bf1
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 4e+01]
Found heuristic solution: objective 500.0000000
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros
Variable types: 0 continuous, 2 integer (0 binary)
Found heuristic solution: objective 550.0000000

Root relaxation: objective 1.033333e+03, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0    

In [75]:
f"{r'.\test_model}\\{model.ModelName}.lp"

SyntaxError: f-string expression part cannot include a backslash (357301512.py, line 1)

In [69]:
RLP = grb.Model('Relaxed MIP')

var_index = [
    f'x_{i}'
    for i in range(1, 3)
]
x_dict = RLP.addVars(var_index, vtype=grb.GRB.CONTINUOUS, name= var_index)
RLP.addConstr(2 * x_dict['x_1'] + x_dict['x_2'] <=10, name= 'c1')
RLP.addConstr(3 * x_dict['x_1'] + 6 * x_dict['x_2'] <=40, name='c2')
RLP.setObjective(100 * x_dict['x_1'] + 150 * x_dict['x_2'], grb.GRB.MAXIMIZE)
# RLP.optimize()

<gurobi.Model Continuous instance Relaxed MIP: 0 constrs, 0 vars, Parameter changes: Username=(user-defined)>

In [49]:
RLP.write(r'.\test_model\simple_IP_model_01.lp')
RLP.write(r'.\test_model\simple_IP_model_01_solution.sol')

In [52]:
RLP.ModelName

'Relaxed MIP'

In [37]:
temp = grb.read('model.lp')

Read LP format model from file model.lp
Reading time = 0.01 seconds
: 2 rows, 2 columns, 4 nonzeros


In [41]:
temp.getVars()[0].Vtype

'C'

In [11]:
coped_rlp=RLP.copy()

In [15]:
coped_rlp.getVars()[0].Vtype

'C'

In [47]:
RLP.setObjective(100 * x_dict['x_1'] + 150 * x_dict['x_2'], grb.GRB.MAXIMIZE)
RLP.Status

1

In [48]:
RLP.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22000.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x60e6e1b1
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 4e+01]
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5000000e+32   2.625000e+30   2.500000e+02      0s
       2    1.0555556e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.055555556e+03


In [52]:
RLP.copy().getVars()[0].vtype

AttributeError: 'gurobipy.Var' object has no attribute 'vtype'

In [45]:
eps = 1e-3
class Global_BB:
    def __init__(self, model):
        self.gub = np.inf
        self.glb = 0
        self.node_pool = []
        
        self.incumbent_solution = None
        self.best_objective = 0
        
        self.MIP_var_vtype_dict = {var.varName: var.vtype for var in model.getVars()}
#         {var_name: 'I' for var_name in [f'x_{i}' for i in range(1, 3)]}
#         {var.varName: var.vtype for var in model.getVars()}
        
        self.model_relaxation = model.copy() 
        for var in self.model_relaxation.getVars():
            var.vtype = grb.GRB.CONTINUOUS
        self.model_relaxation.update()        
        self.root_node = Node(self.model_relaxation, self.MIP_var_vtype_dict, '0')
        
        print('root node begin optimizing')
        self.root_node.optimize_lpr(self.glb)
        print('root optimization finished')
        self.incumbent_solution = self.root_node.x_int_solution
        self.best_objective = self.root_node.node_LB
        self.gub = self.root_node.node_UB
        self.glb = self.root_node.node_LB
        print(f"x_sol:\t{self.root_node.x_solution}\nincumbent_solution:\t{self.incumbent_solution}\nbest_objective:\t{self.best_objective}\nGlobal_UB:\t{self.gub}\nGlobal_LB:\t{self.glb}")
        
        if self.root_node.is_pruned == 0:
            self.node_pool.append(self.root_node)
            self.gub_log = [self.gub]
            self.glb_log = [self.glb]
            print('root node is not punred')
    
    def BandB(self):
#         if len(self.node_pool) == 0:
#             return 
#         node_id = 1
        while len(self.node_pool)>0 and self.gub - self.glb > eps:
            current_node = self.node_pool.pop(self.estimate_node())
            print(f"node: {current_node.node_id} is selected to branch")
            temp = current_node.branch(self)
            printChildNode_info(current_node)
            self.node_pool += temp
            if len(temp) > 0:
                self.gub = max([node.node_UB for node in self.node_pool])
            self.glb_log.append(self.glb)
            self.gub_log.append(self.gub)
            # 更新 全局上下界
            
    
    def estimate_node(self):
#         if len(self.node_pool) == 0:
#             return 1 # no unbranched or pruned node 
        
        # choose the node with greatest node_UB
        best_i = max([
            (i, self.node_pool[i].node_UB)
            for i in range(len(self.node_pool))
        ], key=lambda x: x[-1])[0]
        
        return best_i
    

class Node:
    def __init__(self, model, MIP_var_vtype_dict, node_id):
        self.node_id = node_id
        self.node_LB = 0
        self.node_UB = np.inf
        
        self.x_solution = None
        self.x_int_solution = None
        
        self.MIP_var_vtype_dict = MIP_var_vtype_dict # no relaxation model's var's vtype C for CONTINUOUS; B for BINARY I for INTEGER
        self.branch_var_dict = None
        
        self.model = model
        self.model_status = False
        
        self.is_integer = None
        self.is_pruned = 0 # pruned 1: by integer solution; 2: by bounded 3: by infeasible or unbound
        
        self.branched = False
        
#         self.optimize_lpr()

    
    def optimize_lpr(self, glb):
        self.model.setParam('OutputFlag', 0)
        self.model.update()
        
        # optimize the node model and intialize the node param
        self.model.optimize()
        """
        OPTIMAL = 2
        INFEASIBLE = 3
        UNBOUNDED = 5
        """
        self.model_status = self.model.Status
        
        if self.model_status == 2: # have optimal solution
            self.node_UB = self.model.ObjVal
            
            self.x_solution = {var.varName: var.x for var in self.model.getVars()}
            self.x_int_solution = {var.varName: int(var.x) if self.MIP_var_vtype_dict[var.varName] in ['B', 'I'] else var.x
                                   for var in self.model.getVars()}
#             print(f"x_solution: {self.x_solution}")
#             print(f"x_int_solution: {self.x_int_solution}")
            
            self.node_LB = sum([
                self.x_int_solution[var.varName] * var.Obj
                for var in self.model.getVars()
            ])
            
            
            self.branch_var_dict = {
                var.varName: 1 if (self.MIP_var_vtype_dict[var.varName] in ['B', 'I']
                and abs(self.x_solution[var.varName] - self.x_int_solution[var.varName])>eps) # 1: if a binary or integer var is no integer value; Need to branch
                else 0 # 0: ① if a binary or integer var is integer value; ② continuous var. No Need to branch
                for var in self.model.getVars()
                }
            self.is_integer = 0 if 1 in self.branch_var_dict.values() else 1 # if 1 in branch_var_dict means it still have binary or integer var that is no integer value
            self.PruneByOptimility()
            self.PruneByBounded(glb)
#             self.PruneByOptimility()
#             self.PruneByBounded()
            
        else: # INFEASIBLE or UNBOUNDED      
            self.PrunedByUnfeasibleOrUnbound()
    
    def branch(self, global_BB):
        
        branch_var_list = list(filter(lambda x: self.branch_var_dict[x]==1, self.branch_var_dict))
        # most infeasible branching, 选择小数部分最接近0.5的进行分支
        self.branch_var_name = min(
                [(var_name, abs(self.x_solution[var_name] - self.x_int_solution[var_name])) 
                 for var_name in branch_var_list], key=lambda x : abs(x[-1]-0.5)
        )[0]
        
        # ①BINARY 0, 1. xr=0.xxx  left var BOUND = 0, right_var BOUND =1
        # ②CONTINOUS. xr=A.BBB  left var BOUND= A right var bound = A+1
        
        print(f'branch_var_name:{self.branch_var_name}')
        left_var_bound = int(self.x_solution[self.branch_var_name])
        right_var_bound = left_var_bound+1
        
        # create two child nodes
        self.left_node, self.right_node = self.deepcopy_node(node_id=self.node_id+'->l'), self.deepcopy_node(node_id=self.node_id+'->r')
#         self.left_node.node_id += '->l'
#         self.right_node.node_id += '->r'
        
        # add constraint for left
        branch_var = self.left_node.model.getVarByName(self.branch_var_name)
        self.left_node.model.addConstr(branch_var <= left_var_bound, name='branch_left')
        self.left_node.optimize_lpr(global_BB.glb)
        check_update(global_BB, self.left_node)
    
        
        # add constraint for right
        branch_var = self.right_node.model.getVarByName(self.branch_var_name)
        self.right_node.model.addConstr(branch_var >= right_var_bound, name='branch_right')
        self.right_node.optimize_lpr(global_BB.glb)
        check_update(global_BB, self.right_node)

        
        temp = []
        if self.left_node.is_pruned == 0:
            temp.append(self.left_node)
        if self.right_node.is_pruned == 0:
            temp.append(self.right_node)
        return temp
          
    
    def PruneByOptimility(self):
        if self.is_integer == 1:
#             print('Pruned by optimility')
            self.is_pruned = 1
            
    def PruneByBounded(self, global_lb):
        if self.node_UB < global_lb:
#             print('Pruned by bound')
            self.is_pruned = 2
    
    def PrunedByUnfeasibleOrUnbound(self):
#         print('Pruned by unfeasible or unbound')
        self.is_pruned = 3
    
    def deepcopy_node(self, node_id):
        new_node = Node(self.model.copy(), self.MIP_var_vtype_dict, node_id)
        return new_node

def check_update(global_, local):

    if local.is_pruned != 0: # if node is pruned, pass
        return
    if local.node_LB > global_.glb:
        global_.glb = local.node_LB
        global_.incumbent_solution = local.x_int_solution
        global_.best_objective = local.node_LB
#         return 1

def printChildNode_info(parent):
    is_pruned_dict = {1: 'Pruned by optimility', 2: 'Pruned by bound', 3: 'Pruned by unfeasible or unbound'}
    print(f"node: {parent.node_id} \'child node info:")
    if parent.left_node.is_pruned != 0:
        print(f"left_node: {parent.left_node.node_id} is {is_pruned_dict[parent.left_node.is_pruned]}")
    print(f"left_node {parent.left_node.node_id} info:")
    printNodeInfo(parent.left_node)
    
    if parent.right_node.is_pruned != 0:
        print(f"right_node: {parent.right_node.node_id} is {is_pruned_dict[parent.right_node.is_pruned]}")
    print(f"right_node {parent.right_node.node_id} info:")
    printNodeInfo(parent.right_node)

def printNodeInfo(node):
    print(f"x_sol:\t{node.x_solution}")
    print(f"x_int_sol:\t{node.x_int_solution}")
    print(f"UB:\t{node.node_UB}")
    print(f"LB:\t{node.node_LB}")
    

In [46]:
m = grb.read(r'.\test_model\SimpleIPModel_01.lp')

Read LP format model from file .\test_model\SimpleIPModel_01.lp
Reading time = 0.00 seconds
: 2 rows, 2 columns, 4 nonzeros


In [49]:
m.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22000.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x60e99bf1
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 4e+01]
Found heuristic solution: objective 500.0000000
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros
Variable types: 0 continuous, 2 integer (0 binary)
Found heuristic solution: objective 550.0000000

Root relaxation: objective 1.033333e+03, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0    

In [47]:
test = Global_BB(model=m)

root node begin optimizing
root optimization finished
x_sol:	{'x_1': 2.2222222222222223, 'x_2': 5.555555555555555}
incumbent_solution:	{'x_1': 2, 'x_2': 5}
best_objective:	950.0
Global_UB:	1055.5555555555554
Global_LB:	950.0
root node is not punred


In [48]:
test.BandB()

node: 0 is selected to branch
branch_var_name:x_2
node: 0 'child node info:
left_node 0->l info:
x_sol:	{'x_1': 2.5, 'x_2': 5.0}
x_int_sol:	{'x_1': 2, 'x_2': 5}
UB:	1000.0
LB:	950.0
right_node 0->r info:
x_sol:	{'x_1': 1.3333333333333333, 'x_2': 6.0}
x_int_sol:	{'x_1': 1, 'x_2': 6}
UB:	1033.3333333333333
LB:	1000.0
node: 0->r is selected to branch
branch_var_name:x_1
node: 0->r 'child node info:
left_node 0->r->l info:
x_sol:	{'x_1': 1.0, 'x_2': 6.166666666666667}
x_int_sol:	{'x_1': 1, 'x_2': 6}
UB:	1025.0
LB:	1000.0
right_node: 0->r->r is Pruned by unfeasible or unbound
right_node 0->r->r info:
x_sol:	None
x_int_sol:	None
UB:	inf
LB:	0
node: 0->r->l is selected to branch
branch_var_name:x_2
node: 0->r->l 'child node info:
left_node: 0->r->l->l is Pruned by optimility
left_node 0->r->l->l info:
x_sol:	{'x_1': 1.0, 'x_2': 6.0}
x_int_sol:	{'x_1': 1, 'x_2': 6}
UB:	1000.0
LB:	1000.0
right_node: 0->r->l->r is Pruned by unfeasible or unbound
right_node 0->r->l->r info:
x_sol:	None
x_int_sol:

In [16]:
test.incumbent_solution

{'x_1': 1, 'x_2': 6}

In [12]:
grb.GRB.CONTINUOUS

'C'

In [114]:
IP = grb.Model('IP')
var_index = [
    f'x_{i}'
    for i in range(1, 3)
]
x_dict = IP.addVars(var_index, vtype=grb.GRB.INTEGER, name= var_index)
IP.addConstr(2 * x_dict['x_1'] + x_dict['x_2'] <=10, name= 'c1')
IP.addConstr(3 * x_dict['x_1'] + 6 * x_dict['x_2'] <=40, name='c2')
IP.setObjective(100 * x_dict['x_1'] + 150 * x_dict['x_2'], grb.GRB.MAXIMIZE)
IP.optimize()
IP.getVars()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22000.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x60e99bf1
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 4e+01]
Found heuristic solution: objective 500.0000000
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros
Variable types: 0 continuous, 2 integer (0 binary)
Found heuristic solution: objective 550.0000000

Root relaxation: objective 1.033333e+03, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0    

[<gurobi.Var x_1 (value 1.0)>, <gurobi.Var x_2 (value 6.0)>]

In [80]:
IPr = grb.Model('IP')
var_index = [
    f'x_{i}'
    for i in range(1, 3)
]
x_dict = IPr.addVars(var_index, vtype=grb.GRB.CONTINUOUS, name= var_index)
IPr.addConstr(2 * x_dict['x_1'] + x_dict['x_2'] <=10, name= 'c1')
IPr.addConstr(3 * x_dict['x_1'] + 6 * x_dict['x_2'] <=40, name='c2')
IPr.setObjective(100 * x_dict['x_1'] + 150 * x_dict['x_2'], grb.GRB.MAXIMIZE)
IPr.optimize()
IPr.getVars()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22000.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x60e6e1b1
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 4e+01]
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5000000e+32   2.625000e+30   2.500000e+02      0s
       2    1.0555556e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.055555556e+03


[<gurobi.Var x_1 (value 2.2222222222222223)>,
 <gurobi.Var x_2 (value 5.555555555555555)>]

In [141]:
test = Global_BB(model=IPr)

root node
x_solution: {'x_1': 2.2222222222222223, 'x_2': 5.555555555555555}
x_int_solution: {'x_1': 2, 'x_2': 5}


In [130]:
test.root_node.branch_var_dict

{'x_1': 1, 'x_2': 1}

In [131]:
test.root_node.x_solution

{'x_1': 2.2222222222222223, 'x_2': 5.555555555555555}

In [132]:
test.root_node.x_int_solution

{'x_1': 2, 'x_2': 5}

In [135]:
test.root_node.left_node.node_UB

1000.0

In [136]:
test.root_node.right_node.node_UB

1033.3333333333333

In [137]:
test.root_node.right_node.x_solution

{'x_1': 1.3333333333333333, 'x_2': 6.0}

In [139]:
test.root_node.right_node.right_node.is_pruned

3

In [142]:
test.BandB()

branch_var_name:x_2
x_solution: {'x_1': 2.5, 'x_2': 5.0}
x_int_solution: {'x_1': 2, 'x_2': 5}
x_solution: {'x_1': 1.3333333333333333, 'x_2': 6.0}
x_int_solution: {'x_1': 1, 'x_2': 6}
branch_var_name:x_1
x_solution: {'x_1': 1.0, 'x_2': 6.166666666666667}
x_int_solution: {'x_1': 1, 'x_2': 6}
branch_var_name:x_2
x_solution: {'x_1': 1.0, 'x_2': 6.0}
x_int_solution: {'x_1': 1, 'x_2': 6}
Pruned by optimility
branch_var_name:x_1
x_solution: {'x_1': 2.0, 'x_2': 5.0}
x_int_solution: {'x_1': 2, 'x_2': 5}
Pruned by optimility
Pruned by bound
x_solution: {'x_1': 3.0, 'x_2': 4.0}
x_int_solution: {'x_1': 3, 'x_2': 4}
Pruned by optimility
Pruned by bound


In [ ]:
test.root_node.

In [144]:
test.glb_log

[950.0, 1000.0, 1000.0, 1000.0, 1000.0]

In [145]:
test.gub_log

[1055.5555555555554, 1033.3333333333333, 1025.0, 1025.0, 1025.0]

In [146]:
test.incumbent_solution

{'x_1': 1, 'x_2': 6}

In [ ]:
test.

In [88]:
test.root_node.node_LB

950.0

In [89]:
test.root_node.node_UB

1055.5555555555554

In [71]:
test.root_node.x_int_solution

{}

In [40]:
# {var.varName: var.vtype for var in IP.getVars()}
# IP.getVars()
IP = grb.Model('IP')
var_index = [
    f'x_{i}'
    for i in range(1, 3)
]
x_dict = IP.addVars(var_index, vtype=grb.GRB.INTEGER, name= var_index)
x_dict

{'x_1': <gurobi.Var *Awaiting Model Update*>,
 'x_2': <gurobi.Var *Awaiting Model Update*>}

<gurobi.Model Continuous instance IP: 0 constrs, 0 vars, Parameter changes: Username=(user-defined)>

New model does not contain these changes.
x_solution: {}
x_int_solution: {}
Pruned by optimility


In [27]:
test.MIP_var_vtype_dict

{}

In [72]:
test = grb.Model()
var_index = [
    f'x_{i}'
    for i in range(1, 3)
]
x_dict = test.addVars(var_index, vtype=grb.GRB.CONTINUOUS, name= var_index)
var_index = [
    f'x_{i}'
    for i in range(3, 5)
]
x_dict1 = test.addVars(var_index, vtype=grb.GRB.BINARY, name=var_index)

In [35]:
temp = {'a':1, 'b':0, 'c':0.25, 'd':0.4}
list(filter(lambda x: temp[x]==0, temp))

['b']

In [39]:
temp = (('a',1), ('b',0), ('c',0.25), ('d',0.4))
min(temp, key=lambda x: abs(x[-1]-0.5))

('d', 0.4)